In [53]:
##### MAIN BAYESIAN OPTIMISATION #######

import os
import time
import numpy as np
import yaml
import pickle
from skopt import gp_minimize
from skopt.space import Real, Integer
from bash_config import write_bash  # From your file
import darkfield.mmmUtils_v2 as mu

In [54]:
def generate_yaml(template_path, output_path, param_value, base_index=1):
    with open(template_path) as f:
        ip = yaml.safe_load(f)

    if param_value == 0:
        ip['O2']['in'] = 0
    else:
        ip['O2']['in'] = 1
        ip['O2']['size'] = float(param_value * 1e-6)

    filename = f"BO_{base_index}.yaml" 
    fullpath = os.path.join(output_path, filename)
    with open(fullpath, 'w') as f_out:
        yaml.dump(ip, f_out, sort_keys=False)

    return filename, fullpath



In [55]:
def submit_job(yaml_filename, sim_index, n_sim_points=500, n_cpus=24, mem="600GB"):
    
    bash_dir = "/home/yu79deg/darkfield_p5438/bash"
    
    bash_path = write_bash( path=bash_dir, N=n_sim_points, upd_params={ 'n_cpus': n_cpus, 'mem': mem, 'yaml': yaml_filename } )
    
    os.system(f"sbatch {bash_path}")


In [56]:

def wait_for_completion(jobname, timeout=3600, check_interval=60):
    result_path = f"/home/yu79deg/darkfield_p5438/Aime/pickles/{jobname}_res.pickle"
    waited = 0
    while not os.path.exists(result_path) and waited < timeout:
        print(f"Waiting for job {jobname} to complete...")
        time.sleep(check_interval)
        waited += check_interval
    if not os.path.exists(result_path):
        raise TimeoutError(f"Job {jobname} did not complete in time.")
    print(result_path)
    return result_path



def extract_shadow_factor(pickle_path):
    #result_data = pickle.load(f)
    print(pickle_path)
    result_data = mu.loadPickle(pickle_path) #loading the general parameters
    params=result_data[1]
    Shadow_factor_75 = params['intensities']['roi2'] / params['intensities']['TCC']

    return Shadow_factor_75


In [57]:
history = []

def objective_function_O2size(O2_value, template_path, output_path, base_index):
    yaml_filename, yaml_path = generate_yaml(template_path, output_path, O2_value, base_index)
    jobname = os.path.splitext(yaml_filename)[0]

    submit_job(yaml_filename, sim_index=base_index, n_sim_points=N_SIM_POINTS)

    result_path = wait_for_completion(jobname)
    shadow_factor = extract_shadow_factor(result_path)

    return -np.log10(shadow_factor)  # Maximize -log10(SF)


In [ ]:
############# LAUNCH OPITMISATION ##########

from skopt import gp_minimize
from skopt.space import Real

# Define bounds
search_space = [Integer(0, 240, name='O2_size')]  # Example range

N_SIM_POINTS = 1000 #resolution of the simulations


def skopt_wrapper(x):
    return objective_function_O2size( O2_value=x[0], template_path='/home/yu79deg/darkfield_p5438/yamls/BO_template.yaml',
        output_path='/home/yu79deg/darkfield_p5438/yamls',base_index = len(history) + 1  # unique yaml name
    )

history = []
result = gp_minimize(
    skopt_wrapper,
    dimensions=search_space,
    n_calls=10,
    n_initial_points=3,
    random_state=42
)

Submitted batch job 3862985
Waiting for job BO_1 to complete...
Waiting for job BO_1 to complete...


In [35]:
import pandas as pd

df = pd.DataFrame({
    'O2_size': [x[0] for x in result.x_iters],
    'neg_log10_shadow': result.func_vals,
})
df['shadow_factor'] = 10 ** (-df['neg_log10_shadow'])
df.to_csv("bayes_opt_results.csv", index=False)


In [ ]:
!squeue --me

In [51]:
!scancel 3862979 3862980 3862981 3862982 3862983 3862984